In [1]:
from epipack import StochasticEpiModel


c:\Users\sharm\anaconda3\envs\iibproject\lib\site-packages\epipack\stochastic_epi_models.py:18: UserWarning: Couldn't find the efficient implementation of `SamplableSet` (see github.com/gstonge/SamplableSet). Proceeding with less efficient implementation.
  warnings.warn("Couldn't find the efficient implementation of `SamplableSet` (see github.com/gstonge/SamplableSet). Proceeding with less efficient implementation.")


In [2]:
from epipack import StochasticEpiModel
import numpy as np

N = 3
edge_weight_tuples = [
    (0, 1, 1/3),
    (0, 2, 2/3),
]
directed = False

S, I, R = list("SIR")

model = StochasticEpiModel(
                compartments=[S,I,R],
                N=N,
                edge_weight_tuples=edge_weight_tuples,
                directed=directed,
            )

infection_rate = 3.0
model.set_link_transmission_processes([
        ('I', 'S', infection_rate, 'I', 'I'),
    ])

recovery_rate = 2.0
model.set_node_transition_processes([
        ('I', recovery_rate, 'R'),
    ])

model.set_random_initial_conditions({S: N-1, I: 1})

initial_node_statuses = np.array([
        model.get_compartment_id(I),
        model.get_compartment_id(S),
        model.get_compartment_id(S),
    ])
model.set_node_statuses(initial_node_statuses)

initial_node_statuses = np.zeros(N,dtype=int)
initial_node_statuses[0] = 1
model.set_node_statuses(initial_node_statuses)

N_meas = 10000
N_inf = 0
N_inf_node_2 = 0
for meas in range(N_meas):
    model.set_node_statuses(initial_node_statuses)
    model.simulate(tmax=1e300)
    if model.node_status[1] == model.get_compartment_id(R):
        N_inf += 1
    if model.node_status[2] == model.get_compartment_id(R):
        N_inf_node_2 += 1

print("Node 1 has been infected in", N_inf/N_meas*100, "% of the measurements.")
print("Node 2 has been infected in", N_inf_node_2/N_meas*100, "% of the measurements.")

N_meas = 10000
N_inf = 0
reproduction_number = 0
for meas in range(N_meas):
    model.set_node_statuses(initial_node_statuses)
    t, result = model.simulate(tmax=1e300)
    reproduction_number += result['R'][-1] -1

print("Node 0 has infected", reproduction_number/N_meas, "neighbors on average.")


Node 1 has been infected in 32.85 % of the measurements.
Node 2 has been infected in 50.0 % of the measurements.
Node 0 has infected 0.8324 neighbors on average.


In [3]:
from epipack import StochasticEpiModel
import networkx as nx

k0 = 50
R0 = 2.5
rho = 1
eta = R0 * rho / k0
omega = 1/14
N = int(1e4)
edges = [ (e[0], e[1], 1.0) for e in \
          nx.fast_gnp_random_graph(N,k0/(N-1)).edges() ]

SIRS = StochasticEpiModel(
            compartments=list('SIR'),
            N=N,
            edge_weight_tuples=edges
            )\
        .set_link_transmission_processes([
            ('I', 'S', eta, 'I', 'I'),
        ])\
        .set_node_transition_processes([
            ('I', rho, 'R'),
            ('R', omega, 'S'),
        ])\
        .set_random_initial_conditions({
                                        'S': N-100,
                                        'I': 100
                                       })
t_s, result_s = SIRS.simulate(40)


In [14]:
from epipack import vis 
from epipack import networks
from pyglet.window import key, mouse, Window
layouted_network = networks.get_random_layout(N, edges)
vis.visualize(SIRS, layouted_network, sampling_dt=0.1, config={'draw_links': False})


SyntaxError: invalid syntax (context_managers.py, line 50)

In [5]:
from epipack.vis import visualize
from epipack import (
    StochasticSIRModel,
    get_2D_lattice_links,
    get_grid_layout
)

# define links and network layout
N_side = 100
N = N_side**2
links = get_2D_lattice_links(N_side, periodic=True, diagonal_links=True)
lattice = get_grid_layout(N)

# define model
R0 = 3; recovery_rate = 1/8
model = StochasticSIRModel(N,R0,recovery_rate,
                           edge_weight_tuples=links)
model.set_random_initial_conditions({'I':20,'S':N-20})

sampling_dt = 1

visualize(model,lattice,sampling_dt,
        config={
                 'draw_nodes_as_rectangles':True,
                 'draw_links':False,
               }
          )


SyntaxError: invalid syntax (context_managers.py, line 50)